In [1]:
!rm -rf /kaggle/working/*

In [2]:
import cv2
import random
import os
import math
import numpy as np
import pydicom
import matplotlib.pyplot as plt

from PIL import Image
from skimage import color

import warnings
warnings.filterwarnings("ignore")

input_dir = '/kaggle/input/brain-tumor-progression-tcia/Brain-Tumor-Progression'
output_dir = '/kaggle/working/'

In [3]:
def data_and_mask_to_list(read_dir : str) -> None:
    t1post_images = []
    mask_tumor_image = []
    
    for folder in os.listdir(read_dir):
        if folder.find("T1post") > -1:
            t1post_images_path = [ read_dir + '/' + folder + '/' + image for image in os.listdir(read_dir + '/' + folder) ]
        elif folder.find("MaskTumor") > -1:
            mask_tumor_images_path = [ read_dir + '/' + folder + '/' + image for image in os.listdir(read_dir + '/' + folder) ]

    return list(zip(t1post_images_path, mask_tumor_images_path))

In [4]:
def create_needed_folders(parent_folder_name: str) -> None:
    os.makedirs(parent_folder_name + '/' + 'Train')
    os.makedirs(parent_folder_name + '/' + 'Train/' + 'Mask')
    os.makedirs(parent_folder_name + '/' + 'Train/' + 'Image')
    
    os.makedirs(parent_folder_name + '/' + 'Test')
    os.makedirs(parent_folder_name + '/' + 'Test/' + 'Mask')
    os.makedirs(parent_folder_name + '/' + 'Test/' + 'Image')   
    
    os.makedirs(parent_folder_name + '/' + 'Valid')
    os.makedirs(parent_folder_name + '/' + 'Valid/' + 'Mask')
    os.makedirs(parent_folder_name + '/' + 'Valid/' + 'Image')

In [5]:
patients = [p for p in os.listdir(input_dir)]
patients.remove('LICENSE')

data_mask_path_list = []

for patient in patients:
    os.makedirs(output_dir + patient)
    create_needed_folders(output_dir + patient)
    for dir in os.listdir(input_dir + '/' + patient):
        data_mask_path_list.extend(data_and_mask_to_list(input_dir + "/" + patient + "/" + dir))

random.shuffle(data_mask_path_list)

train_data_path_list = data_mask_path_list[: int(len(data_mask_path_list)  * 0.5)]
test_data_path_list = data_mask_path_list[int(len(data_mask_path_list) * 0.5) : int(len(data_mask_path_list) * 0.8)]
valid_data_path_list = data_mask_path_list[int(len(data_mask_path_list) * 0.8) :]

trainX, trainY = [x[0] for x in train_data_path_list], [y[1] for y in train_data_path_list]
testX, testY = [x[0] for x in test_data_path_list], [y[1] for y in test_data_path_list]
validX, validY = [x[0] for x in valid_data_path_list], [y[1] for y in valid_data_path_list]

In [6]:
def DICOM_to_PNG(dataset: str) -> None:
    for case in dataset:
        patient = case[case.find('PGBM-0'): case.find('PGBM-0') + 8]

        folder = None
        if case in trainX or case in trainY:
            folder = 'Train'
        elif case in testX or case in testY:
            folder = 'Test'
        elif case in validX or case in validY:
            folder = 'Valid'
        
        if case.find('T1post') != -1:
            folder += '/Image'
        else:
            folder += '/Mask'

        name = case[case.find('.dcm') - 4: case.find('.dcm')]
        img = pydicom.dcmread(case).pixel_array.astype(float)
        scaled_img = np.uint8((np.maximum(img, 0) / img.max()) * 255)
        cv2.imwrite(output_dir + patient + '/' + folder + '/' + name + '.png', scaled_img)

In [7]:
DICOM_to_PNG(trainX)
DICOM_to_PNG(trainY)
DICOM_to_PNG(testX)
DICOM_to_PNG(testY)
DICOM_to_PNG(validX)
DICOM_to_PNG(validY)

In [8]:
import torch
import torchvision

print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())

import sys

!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/sam2.git'

!mkdir -p ../checkpoints/
!wget -P ../checkpoints/ https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt

PyTorch version: 2.4.0
Torchvision version: 0.19.0
CUDA is available: True
  Cloning https://github.com/facebookresearch/sam2.git to /tmp/pip-req-build-q396yl4z
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/sam2.git /tmp/pip-req-build-q396yl4z
  Resolved https://github.com/facebookresearch/sam2.git to commit 2b90b9f5ceec907a1c18123530e92e794ad901a4
  Installing build dependencies ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ 